In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd

In [51]:
def draft_year_to_season_string(year):
    start_year = year - 1
    end_year = str(year)[-2:]
    return f"{start_year}-{end_year}"

def get_per_game_stats(player_name, target_season):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    
    try:
        driver.get("https://www.sports-reference.com/cbb/players/")

        # wait for search bar to appear
        search_bar = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "input[name='search']"))
        )
        search_bar.clear()
        search_bar.send_keys(player_name)

        search_bar.send_keys(Keys.RETURN)
        time.sleep(3)

        # parse player page with beautifulsoup
        soup = BeautifulSoup(driver.page_source, "html.parser")
        table = soup.find("table", id="players_per_game")
        if table is None:
            print(f"No per-game stats found for {player_name}")

        # turn into pandas df
        df = pd.read_html(str(table))[0]
        df = df[df['Season'] == draft_year_to_season_string(target_season)]
        df = df.drop(columns=['Team', 'Conf', 'Class', 'Awards'])
        df['Name'] = player_name

        # reorder columns
        cols = df.columns.tolist()
        cols.remove("Name")
        df = df[["Name"] + cols]

        return df.reset_index(drop=True)

    finally:
        driver.quit()
        

In [52]:
# testing 
get_per_game_stats("Zach Edey", 2024)

/var/folders/6l/0dr5mry10lddprxw6f8y9s3w0000gn/T/ipykernel_78462/2521058996.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


,Name,Season,Pos,G,GS,MP,FG,FGA,FG%,3P,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Zach Edey,2023-24,C,39,39,32.0,8.6,13.8,0.623,0.0,...,0.711,4.6,7.5,12.2,2.0,0.3,2.2,2.3,1.9,25.2
